In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from top2vec import Top2Vec  # https://paperswithcode.com/paper/top2vec-distributed-representations-of-topics

import umap

In [2]:
path_corpora = Path('data/corpora')

# path_corpus_tut = path_corpora / Path('tutby_126784.csv')
path_corpus_tut = path_corpora / Path('clean_tutby_126784.csv')
# path_corpus_medicine = path_corpora / Path('medicine.csv')
# path_corpus_telegram = path_corpora / Path('telegram')

# path_model_top2vec = Path('data/model/top2vec_126784.joblibfile')
path_model_top2vec = Path('data/model/top2vec_clean_126784.joblibfile')
# path_points = Path('data/model/points_126784.npy')
path_points = Path('data/model/points_clean_126784.npy')

In [3]:
data = pd.read_csv(path_corpus_tut)
display(data.head(3))

df0 = data['document']
# df1 = pd.read_csv(path_corpus_medicine)['document']
# dfs = [pd.read_csv(path)['document'] for path in path_corpus_telegram.iterdir()]

# corpus = pd.concat((df0, df1, *dfs))
corpus = df0
corpus = corpus.fillna('')

print(corpus.shape)

,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']


(126784,)


In [5]:
%%time

model = Top2Vec(corpus, speed='learn', workers=2, verbose=True)


model.save(path_model_top2vec)

2020-09-26 00:46:30,054 - top2vec - INFO - Pre-processing documents for training
2020-09-26 00:48:44,730 - top2vec - INFO - Creating joint document/word embedding
2020-09-26 03:48:06,927 - top2vec - INFO - Creating lower dimension embedding of documents
c:\users\tim\documents\github\oxagile\nn_venv\lib\site-packages\umap\nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "..\..\github\oxagile\nn_venv\lib\site-packages\umap\utils.py", line 409:
@numba.njit(parallel=True)
def build_candidates(current_graph, n_vertices, n_neighbors, max_candidates, rng_state):
^

  current_graph, n_vertices, n_neighbors, max_candidates, rng_state
c:\users\tim\documents\github\oxagile\nn_venv\lib\site-packages\numba\typed_passes.py:293: NumbaPerformanceWarning: 

Wall time: 3h 6min 27s


In [6]:
%%time

model = Top2Vec.load(path_model_top2vec)

model_umap = umap.UMAP(n_neighbors=15, n_components=2, metric='cosine')

points = model_umap.fit_transform(model.model.docvecs.vectors_docs)

with open(path_points, 'wb') as file:
    np.save(file, points)

print(points.shape)

c:\users\tim\documents\github\oxagile\nn_venv\lib\site-packages\numba\typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "..\..\github\oxagile\nn_venv\lib\site-packages\umap\nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  state.func_ir.loc))


(126784, 2)
Wall time: 3min 45s


In [ ]:
# umap_model = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine').fit(embeddings)
# cluster = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom').fit(umap_model.embedding_)
# labels = cluster.labels_